In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from ast import literal_eval

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
df_movies = pd.read_csv('movie_data.csv',engine='python')
df_ratings = pd.read_csv('ratings_export.csv', engine='python')

In [5]:
display(df_movies.shape)
display(df_ratings.shape)

(285963, 19)

(11078167, 4)

In [6]:
# df_movies.info()

need = ['genres','movie_id','movie_title','original_language','year_released','tmdb_link']

In [7]:
df_movies = df_movies.dropna(subset=['movie_title'])
df_movies = df_movies.dropna(subset=['movie_id'])
df_movies = df_movies.dropna(subset=['genres'])
df_movies = df_movies.loc[df_movies['genres'] != '[]']
df_movies = df_movies.loc[:,need]

In [8]:
df_movies.loc[df_movies['original_language'] == 'or']

,genres,movie_id,movie_title,original_language,year_released,tmdb_link
46612,"[""Family"",""Science Fiction"",""Comedy""]",the-friend-1985,The Friend,or,"1,985.00",https://www.themoviedb.org/movie/579691/
157062,"[""Drama""]",shesha-drushti,Shesha Drushti,or,"1,997.00",https://www.themoviedb.org/movie/465637/
243030,"[""Drama""]",maya-miriga,Maya Miriga,or,"1,984.00",https://www.themoviedb.org/movie/392292/
256593,"[""Drama""]",yesterdays-past,Yesterday's Past,or,"2,020.00",https://www.themoviedb.org/movie/767661/


In [9]:
df_ratings = df_ratings.dropna(subset = ['movie_id'])
# df_ratings.isna().sum()

In [10]:
# plt.boxplot(df_ratings['rating_val'])
# plt.show()
avg_rating = df_ratings.groupby('movie_id')['rating_val'].median().to_frame().reset_index()
count_users = df_ratings.groupby('movie_id')['user_id'].count().to_frame().reset_index()

In [11]:
joint_table = pd.merge(left=avg_rating,right=count_users,on='movie_id',how='inner')
display(joint_table.head())

,movie_id,rating_val,user_id
0,0-uhr-15-zimmer-9,5.00,2
1,00-00,1.00,3
2,00-08,6.00,19
3,00-schneider-im-wendekreis-der-eidechse,7.00,15
4,00-schneider-jagd-auf-nihil-baxter,6.00,25


In [12]:
joint_table['popularity_index'] = joint_table['rating_val']*joint_table['user_id']
joint_table.sort_values(by='popularity_index',ascending=False,inplace=True)


In [13]:
joint_table.drop(labels=['rating_val','user_id'],axis=1,inplace=True)
# joint_table.head()

In [14]:
movie_list = pd.merge(left=joint_table,right=df_movies,on='movie_id',how='left')
# display(movie_list.head(20))

In [15]:
movie_list.dropna(subset=['genres'],inplace = True)

In [16]:
movie_list['genres'] = movie_list['genres'].apply(literal_eval)
i = input('Which genre do you want to check for: ')
def genre_rank(i,x):
    # x = literal_eval(x)
    for val in x:
        if i in val:
            flag = (x.index(val))+1
            break
        else:
            flag = 0
    return flag

movie_list[f'has_{i}'] = movie_list['genres'].apply(lambda x: genre_rank(i,x))
# display(movie_list[f'has_{i}'].value_counts())

In [17]:
# movie_list.drop(labels=['rating_val','user_id','popularity','tmdb_id'],axis=1,inplace=True)
movie_list.head()

,movie_id,popularity_index,genres,movie_title,original_language,year_released,tmdb_link,has_Thriller
0,parasite-2019,"59,980.00","[Comedy, Thriller, Drama]",Parasite,ko,"2,019.00",https://www.themoviedb.org/movie/496243/,2
1,spider-man-into-the-spider-verse,"47,610.00","[Action, Adventure, Animation, Science Fiction...",Spider-Man: Into the Spider-Verse,en,"2,018.00",https://www.themoviedb.org/movie/324857/,0
2,the-shining,"45,882.00","[Horror, Thriller]",The Shining,en,"1,980.00",https://www.themoviedb.org/movie/694/,2
3,the-grand-budapest-hotel,"45,792.00","[Comedy, Drama]",The Grand Budapest Hotel,en,"2,014.00",https://www.themoviedb.org/movie/120467/,0
4,the-dark-knight,"45,504.00","[Drama, Action, Crime, Thriller]",The Dark Knight,en,"2,008.00",https://www.themoviedb.org/movie/155/,4


In [18]:
movie_list = movie_list[['movie_id','movie_title','original_language','popularity_index','genres',f'has_{i}',
                         'year_released','tmdb_link']]

In [19]:
movie_list.loc[(movie_list[f'has_{i}'] != 0) & (movie_list['year_released'] > 1999) & 
               ((movie_list['original_language'] == 'bn'))].sort_values(by=[f'has_{i}','popularity_index'],
                                            ascending=[True,False]).to_excel(f'movie_has_{i}_list.xlsx')